In [1]:
from sqlalchemy import create_engine, Column, String, Integer, Boolean, Date
from sqlalchemy.orm import declarative_base, sessionmaker
import pandas as pd
from tqdm import tqdm
import sys

sys.path.append("..")
import os
from dotenv import load_dotenv
import financedatabase as fd
from investorkit.investorkit.get_data.base import (
    get_profile,
)  # Make sure you have this function or replace it with your own

# Load environment variables
load_dotenv()
pd.set_option("display.max_columns", None)

# Database setup
DATABASE_URL = os.getenv("DATABASE_URL")
engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

In [2]:
# Define Table Schema
Base = declarative_base()


class Profile(Base):
    __tablename__ = "profiles"
    symbol = Column(String, primary_key=True, index=True)
    companyName = Column(String)
    cik = Column(Integer)
    exchange = Column(String)
    exchangeShortName = Column(String)
    industry = Column(String)
    sector = Column(String)
    country = Column(String)
    ipoDate = Column(Date)
    defaultImage = Column(Boolean)
    isEtf = Column(Boolean)
    isActivelyTrading = Column(Boolean)


Base.metadata.create_all(bind=engine)

# Fetch FMP API key
FMP_API_KEY = os.getenv("FMP_SECRET_KEY")


# Fetch Equity Symbols
def fetch_equity_symbols(country="United States", market="NASDAQ Global Select"):
    equities = fd.Equities()
    selected_columns = [
        "name",
        "currency",
        "sector",
        "industry_group",
        "industry",
        "exchange",
        "market",
        "market_cap",
    ]
    us_equities = equities.select(country=country)
    df_equities = us_equities[us_equities["market"] == market][selected_columns]
    list_symbols = list(df_equities.index)
    return list_symbols

In [3]:
# Get New Symbols
def get_new_symbols(list_symbols, engine):
    existing_symbols_query = "SELECT symbol FROM profiles;"
    existing_symbols = pd.read_sql(existing_symbols_query, con=engine)
    new_symbols = list(set(list_symbols) - set(existing_symbols["symbol"].tolist()))
    return new_symbols


# Store Profiles in DB
def store_profiles(df_profiles_filtered, engine):
    df_profiles_filtered.to_sql("profiles", con=engine, if_exists="append", index=False)

In [4]:
list_symbols = fetch_equity_symbols()

In [7]:
# Check for new symbols
new_symbols = get_new_symbols(list_symbols, engine)
new_symbols = new_symbols[:10]  # For testing purposes


# If there are new symbols, fetch and store their profiles
if new_symbols:
    df_profiles = get_profile(new_symbols, FMP_API_KEY)
    list_cols = [
        "symbol",
        "companyName",
        "cik",
        "exchange",
        "exchangeShortName",
        "industry",
        "sector",
        "country",
        "ipoDate",
        "defaultImage",
        "isEtf",
        "isActivelyTrading",
    ]
    df_profiles_filtered = df_profiles[list_cols]

    # Replace empty strings in 'ipoDate' with None
    df_profiles_filtered["ipoDate"].replace("", None, inplace=True)

    store_profiles(df_profiles_filtered, engine)

100%|██████████| 10/10 [00:06<00:00,  1.56it/s]
/tmp/ipykernel_108691/1549707433.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_profiles_filtered['ipoDate'].replace('', None, inplace=True)


In [8]:
# Fetch the first 10 records from the 'profiles' table
query = "SELECT * FROM profiles LIMIT 10;"
df = pd.read_sql(query, engine)

  symbol                                        companyName      cik  \
0  LANDO                         Gladstone Land Corporation  1495240   
1   ARCC                           Ares Capital Corporation  1287750   
2   PBHC                           Pathfinder Bancorp, Inc.  1609065   
3   ESSC                 East Stone Acquisition Corporation  1760683   
4  SAMAW      Schultze Special Purpose Acquisition Corp. II  1843100   
5   GVCI  Green Visor Financial Technology Acquisition C...  1858503   
6   AHAC                 Alpha Healthcare Acquisition Corp.  1818382   
7   IBRX                                  ImmunityBio, Inc.  1326110   
8   SNAX                                 Stryve Foods, Inc.  1691936   
9  PRSTW                            Presto Automation, Inc.  1822145   

                exchange exchangeShortName              industry  \
0   NASDAQ Global Market            NASDAQ       REIT—Industrial   
1   NASDAQ Global Market            NASDAQ      Asset Management   
2  

In [9]:
df

,symbol,companyName,cik,exchange,exchangeShortName,industry,sector,country,ipoDate,defaultImage,isEtf,isActivelyTrading
0,LANDO,Gladstone Land Corporation,1495240,NASDAQ Global Market,NASDAQ,REIT—Industrial,Real Estate,US,2020-10-19,False,False,True
1,ARCC,Ares Capital Corporation,1287750,NASDAQ Global Market,NASDAQ,Asset Management,Financial Services,US,2004-10-05,False,False,True
2,PBHC,"Pathfinder Bancorp, Inc.",1609065,NASDAQ Capital Market,NASDAQ,Banks—Regional,Financial Services,US,1995-11-16,False,False,True
3,ESSC,East Stone Acquisition Corporation,1760683,NASDAQ Capital Market,NASDAQ,Shell Companies,Financial Services,CN,2020-02-14,True,False,False
4,SAMAW,Schultze Special Purpose Acquisition Corp. II,1843100,NASDAQ Global Market,NASDAQ,Shell Companies,Financial Services,US,2019-01-09,False,False,True
5,GVCI,Green Visor Financial Technology Acquisition C...,1858503,NASDAQ Global Market,NASDAQ,Shell Companies,Financial Services,US,2021-12-30,False,False,False
6,AHAC,Alpha Healthcare Acquisition Corp.,1818382,NASDAQ Global Select,NASDAQ,Shell Companies,Financial Services,US,2020-11-09,False,False,False
7,IBRX,"ImmunityBio, Inc.",1326110,NASDAQ Global Select,NASDAQ,Biotechnology,Healthcare,US,2015-07-28,False,False,True
8,SNAX,"Stryve Foods, Inc.",1691936,NASDAQ Capital Market,NASDAQ,Packaged Foods,Consumer Defensive,US,2019-03-06,False,False,True
9,PRSTW,"Presto Automation, Inc.",1822145,NASDAQ Global Market,NASDAQ,Software—Application,Technology,US,None,True,False,True


In [6]:


# Main Execution
def main():
    # Fetch the equity symbols
   
if __name__ == "__main__":
    main()


NameError: name 'tqdm' is not defined

In [3]:
from sqlalchemy import create_engine, Column, String, Integer, Boolean, Date
from sqlalchemy.orm import declarative_base

from sqlalchemy.orm import sessionmaker

import sys
from dotenv import load_dotenv
import os
import financedatabase as fd

from investorkit.investorkit.get_data.base import get_profile, get_financial_statements

load_dotenv()

sys.path.append("..")
import pandas as pd

pd.set_option("display.max_columns", None)

DATABASE_URL = os.getenv("DATABASE_URL")

# Database connection

engine = create_engine(DATABASE_URL)
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Table schema
Base = declarative_base()


class Profile(Base):
    __tablename__ = "profiles"

    symbol = Column(String, primary_key=True, index=True)
    companyName = Column(String)
    cik = Column(Integer)
    exchange = Column(String)
    exchangeShortName = Column(String)
    industry = Column(String)
    sector = Column(String)
    country = Column(String)
    ipoDate = Column(Date)
    defaultImage = Column(Boolean)
    isEtf = Column(Boolean)
    isActivelyTrading = Column(Boolean)


Base.metadata.create_all(bind=engine)


FMP_API_KEY = os.getenv("FMP_SECRET_KEY")

# Questo blocco diventa una funzione che prende in input una serie di parametri che servono per filtrare la lista dei ticker da considerare. Al momento come parametro consideriamo
# solo il country (default = "United States") e il market (default = "NASDAQ Global Select")
equities = fd.Equities()
us_equities = equities.select(country="United States")
selected_columns = [
    "name",
    "currency",
    "sector",
    "industry_group",
    "industry",
    "exchange",
    "market",
    "market_cap",
]
df_equities = us_equities[us_equities["market"] == "NASDAQ Global Select"][
    selected_columns
]

list_symbols = list(df_equities.index)
# Verifichiamo quali sono i ticker che non hanno un profilo su FMP facendo il check di quali ticker sono prensenti nella tabella profile
# Bisogna quindi prima fare il chek che il database esiste e che la tabella esiste. Se non esiste bisogna creare il database e la tabella.
# La tabella profile deve essere creata con i seguenti campi:
# symbol, companyName, cik, exchange, exchangeShortName, industry, sector, country, ipoDate, defaultImage, isEtf, isActivelyTrading
# che hanno le seguenti caratteristiche:
# <class 'pandas.core.frame.DataFrame'>
# RangeIndex: 3 entries, 0 to 2
# Data columns (total 12 columns):
#  #   Column             Non-Null Count  Dtype
# ---  ------             --------------  -----
#  0   symbol             3 non-null      object
#  1   companyName        3 non-null      object
#  2   cik                3 non-null      int64
#  3   exchange           3 non-null      object
#  4   exchangeShortName  3 non-null      object
#  5   industry           2 non-null      object
#  6   sector             3 non-null      object
#  7   country            2 non-null      object
#  8   ipoDate            3 non-null      object
#  9   defaultImage       3 non-null      bool
#  10  isEtf              3 non-null      bool
#  11  isActivelyTrading  3 non-null      bool
# dtypes: bool(3), int64(1), object(8)
# memory usage: 353.0+ bytes

# Verranno quindi estratti solo i dati reltivi alla lista di ticker list_symbols_new che non sono presenti nel database (inizialmente la tabella è vuota)
# Genera la funzione che ritorna list_symbols_new

df_profiles = get_profile(list_symbols_new, FMP_API_KEY)

list_cols = [
    "symbol",
    "companyName",
    "cik",
    "exchange",
    "exchangeShortName",
    "industry",
    "sector",
    "country",
    "ipoDate",
    "defaultImage",
    "isEtf",
    "isActivelyTrading",
]

df_profiles_filtered = df_profiles[list_cols]

# devono quindi essere storicizzati i nuovi dati presenti in df_profiles_filtered

In [4]:
df_profiles_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   symbol             3 non-null      object
 1   companyName        3 non-null      object
 2   cik                3 non-null      int64 
 3   exchange           3 non-null      object
 4   exchangeShortName  3 non-null      object
 5   industry           2 non-null      object
 6   sector             3 non-null      object
 7   country            2 non-null      object
 8   ipoDate            3 non-null      object
 9   defaultImage       3 non-null      bool  
 10  isEtf              3 non-null      bool  
 11  isActivelyTrading  3 non-null      bool  
dtypes: bool(3), int64(1), object(8)
memory usage: 353.0+ bytes


100%|██████████| 100/100 [00:56<00:00,  1.78it/s]


In [ ]:
# df_equities viene storicizzato in una tabella
# In una tabella di log voglio storicizzare un record che mi dice:
# - numero di ticker considerati
# - numero di nuovi ticker

In [8]:
df_income, invalid_tickers_income = get_financial_statements(
    tickers=["AAPL", "META"], statement="income", api_key=FMP_API_KEY
)
df_balance, invalid_tickers_balance = get_financial_statements(
    tickers=["AAPL", "META"], statement="balance", api_key=FMP_API_KEY
)
df_cashflow, invalid_tickers_cashflow = get_financial_statements(
    tickers=["AAPL", "META"], statement="cashflow", api_key=FMP_API_KEY
)

Obtaining income data:   0%|          | 0/2 [00:00<?, ?it/s]

Obtaining income data: 100%|██████████| 2/2 [00:02<00:00,  1.11s/it]


In [ ]:
sel_cols = [
    "date",
    "symbol",
    "reportedCurrency",
    "cik",
    "fillingDate",
    "acceptedDate",
    "calendarYear",
    "period",
    "revenue",
    "costOfRevenue",
    "grossProfit",
    "grossProfitRatio",
    "researchAndDevelopmentExpenses",
    "generalAndAdministrativeExpenses",
    "sellingAndMarketingExpenses",
    "sellingGeneralAndAdministrativeExpenses",
    "otherExpenses",
    "operatingExpenses",
    "costAndExpenses",
    "interestIncome",
    "interestExpense",
    "depreciationAndAmortization",
    "ebitda",
    "ebitdaratio",
    "operatingIncome",
    "operatingIncomeRatio",
    "totalOtherIncomeExpensesNet",
    "incomeBeforeTax",
    "incomeBeforeTaxRatio",
    "incomeTaxExpense",
    "netIncome",
    "netIncomeRatio",
    "eps",
    "epsdiluted",
    "weightedAverageShsOut",
    "weightedAverageShsOutDil",
    "link",
    "finalLink",
]

In [ ]:
list_ticker = list_ticker_nasdaq

In [ ]:
df_log = pd.DataFrame(
    [
        {
            "timestamp": datetime.now(),
            "num_companies": len(companies),
            "num_tickers": len(list_ticker),
        }
    ]
)

In [ ]:
bulk_insert_data_from_dataframe(LogAvailableTickers, df_log)

In [ ]:
# Partiamo da una lista di ticker ridotta e iteriamo su 100 trimestri. Sicuramente c'è un modo piu ottimale di farlo, ma al momento lo lasciamo cosi
list_ticker = [
    "AAPL",
    "PLTR",
    "AMZN",
    "META",
    "NIO",
    "RGTI",
    "FFIEW",
    "NVDA",
    "NFLX",
    "BABA",
]

source = "FinancialModelingPrep"
quarterly = True
limit = 100

# Leggiamo i dati già storicizzati
df = read_table_data("fa_balance")

In [ ]:
df

,fiscal_date_ending,reported_currency,cik,filling_date,accepted_date,calendar_year,period,cash_and_cash_equivalents,short_term_investments,cash_and_short_term_investments,...,total_liabilities_and_stockholders_equity,minority_interest,total_liabilities_and_total_equity,total_investments,total_debt,net_debt,link,final_link,ticker,current_date


In [ ]:
def process_ticker(ticker, df):
    df_quarterly = openbb.stocks.fa.balance(
        symbol=ticker, source=source, quarterly=quarterly, limit=limit
    )

    if df_quarterly.empty:
        print(f"Ticker {ticker} not found")
        return df, None

    df_quarterly = df_quarterly.T.reset_index()
    df_quarterly["ticker"] = ticker
    df_quarterly["current_date"] = pd.Timestamp.now().floor("S")
    df_quarterly.columns = [
        col.lower().replace(" ", "_") for col in df_quarterly.columns
    ]

    # Convert both 'calendar_year' columns to datetime
    df_quarterly["calendar_year"] = pd.to_datetime(df_quarterly["calendar_year"])
    if "calendar_year" in df.columns:
        df["calendar_year"] = pd.to_datetime(df["calendar_year"])

    if df.empty:
        df_log_process = pd.DataFrame(
            [
                {
                    "ingestion_timestamp": datetime.now(),
                    "ticker": ticker,
                    "num_new_rows": len(df_quarterly),
                    "num_old_rows_diff_dates": 0,
                    "num_columns": len(df_quarterly.columns),
                }
            ]
        )

        return df_quarterly, df_log_process

    df_quarterly_merge = df_quarterly.merge(
        df,
        how="left",
        indicator=True,
        on=["cik", "calendar_year", "period"],
        suffixes=("", "_y"),
    )
    df_new_rows = df_quarterly_merge[df_quarterly_merge["_merge"] == "left_only"]
    df_old_rows = df_quarterly_merge[df_quarterly_merge["_merge"] == "both"]
    df_old_rows_diff_dates = df_old_rows[
        (df_old_rows["filling_date"] != df_old_rows["filling_date_y"])
        | (df_old_rows["accepted_date"] != df_old_rows["accepted_date_y"])
    ]

    df_log_process = pd.DataFrame(
        [
            {
                "ingestion_timestamp": datetime.now(),
                "ticker": ticker,
                "num_new_rows": len(df_new_rows),
                "num_old_rows_diff_dates": len(df_old_rows_diff_dates),
                "num_columns": len(df_quarterly.columns),
            }
        ]
    )

    return (
        pd.concat([df_new_rows[df.columns], df_old_rows_diff_dates[df.columns]]),
        df_log_process,
    )

In [ ]:
# Main script
dataframes = []
records = []

for ticker in list_ticker:
    df_processed, df_log = process_ticker(ticker, df)
    if df_processed is not None:
        dataframes.append(df_processed)
    if df_log is not None:
        bulk_insert_data_from_dataframe(LogIngestedTickers, df_log)

df_final = pd.concat(dataframes, ignore_index=True)

date_columns = ["filling_date", "accepted_date", "current_date"]
for col in date_columns:
    df_final[col] = pd.to_datetime(df_final[col])

INFO:openbb_terminal.stocks.fundamental_analysis.sdk_helpers:START
INFO:openbb_terminal.stocks.fundamental_analysis.sdk_helpers:{"INPUT": {"quarterly": "True", "ratios": "False", "source": "FinancialModelingPrep", "limit": "100", "symbol": "AAPL", "chart": "False"}, "VIRTUAL_PATH": "stocks.fa.balance", "CHART": false}
INFO:openbb_terminal.stocks.fundamental_analysis.fmp_model:START
INFO:openbb_terminal.stocks.fundamental_analysis.fmp_model:START
INFO:openbb_terminal.stocks.fundamental_analysis.fmp_model:END
INFO:openbb_terminal.stocks.fundamental_analysis.fmp_model:END
INFO:openbb_terminal.stocks.fundamental_analysis.sdk_helpers:END
INFO:openbb_terminal.stocks.fundamental_analysis.sdk_helpers:START
INFO:openbb_terminal.stocks.fundamental_analysis.sdk_helpers:{"INPUT": {"quarterly": "True", "ratios": "False", "source": "FinancialModelingPrep", "limit": "100", "symbol": "PLTR", "chart": "False"}, "VIRTUAL_PATH": "stocks.fa.balance", "CHART": false}
INFO:openbb_terminal.stocks.fundamental

In [ ]:
df_final["ticker"].value_counts()

AAPL     100
AMZN     100
NVDA      98
NFLX      86
META      47
BABA      43
NIO       23
PLTR      14
FFIEW     13
RGTI      11
Name: ticker, dtype: int64

In [ ]:
bulk_insert_data_from_dataframe(Balance, df_final)